# LOG6302A — Analyse d’applications et Cyber-sécurité<br>Laboratoire #4

**Quentin Guidée (2206809), Nam Vu (2230468)**

Polytechnique Montréal – Hiver 2024


In [ ]:
import tempfile
from copy import deepcopy

from code_analysis import CFG, CFGReader, Graph
from graphviz import Source
from IPython.display import Image
from ptfa import PossiblyReachingDataFlow, PossiblyReachableDataFlow

cfg_reader = CFGReader()


def show_graph(graph: Graph):
    """Afficher le graphe dans Jupyter"""
    dot = graph.to_dot()
    s = Source(dot)
    with tempfile.NamedTemporaryFile(suffix=".png") as f:
        s.render(f.name, format="png")
        display(Image(f.name + ".png"))


def show_in_out(cfg: CFG, in_dict: dict[int, bool], out_dict: dict[int, bool]) -> CFG:
    """Annote un CFG avec les valeurs de in et out pour chaque noeud puis l'affiche"""
    cfg_copy = deepcopy(cfg)
    for nid in cfg_copy.get_node_ids():
        cfg_copy.set_image(
            nid,
            f"in={in_dict[nid]} out={out_dict[nid]} {cfg_copy.get_image(nid) or ''}",
        )
    show_graph(cfg_copy)
    return cfg_copy